<a href="https://colab.research.google.com/github/donghuna/PromptGenerate/blob/main/Evaluation-gsm8k_plan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset
from types import FunctionType
import re
from tqdm import tqdm

In [3]:
gsm8k_dataset = load_dataset("donghuna/generated_code-gsm8k-Agent-A")
gsm8k_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/378 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/861k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1001 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'plan', 'code'],
        num_rows: 1001
    })
})

In [4]:
def execute_code(code):


    try:
        return exec(code, globals())

        # check_func = globals()['check']

        # candidate_func = globals().get(entry_point)
        # if not isinstance(candidate_func, FunctionType):
        #     raise ValueError(f"{entry_point} is not a function")

        # # check 함수로 검증 실행
        # check_func(candidate_func)
        return True
    # except AssertionError:
    #     print("Test failed: The generated solution did not pass the tests.")
    #     return False
    # except KeyError as e:
    #     print(f"Error: Missing required function or variable - {e}")
    #     return False
    # except ValueError as ve:
    #     print(f"Error: {ve}")
    #     return False
    # except NameError as ne:
    #     print(f"Error: NameError - {ne}")
    #     return False
    except Exception as e:
        return str(e)

In [14]:
def remove_code_blocks(input_text):
    """
    Remove code block markers (e.g., ```python and ```) from the given text.

    Args:
        input_text (str): The input text containing code blocks.

    Returns:
        str: The text with code block markers removed.
    """
    lines = input_text.splitlines()
    inside_code_block = False
    result = []

    for line in lines:
        # Check for code block markers
        if line.strip().startswith("```python") or line.strip() == "```":
            inside_code_block = not inside_code_block
        elif inside_code_block:
            result.append(line)

    return "\n".join(result)


# # Example usage
# input_text = """
# Some text outside the code block.

# ```python
# # This is a sample Python code
# print("Hello, world!")
# a = 5 + 3


def extract_code(input_text):
    """
    Extract Python code from a mixed text input.
    Keeps plain code as is and removes block markers like ```python and ```.

    Args:
        input_text (str): The input text containing code and other text.

    Returns:
        str: The cleaned code, preserving plain code and extracting code blocks.
    """
    lines = input_text.splitlines()
    inside_code_block = False
    result = []

    for line in lines:
        # Check for code block markers
        if line.strip().startswith("```python") or line.strip() == "```":
            inside_code_block = not inside_code_block

            # print('yes!\n')
            # print(line)
        elif inside_code_block and line.strip():  # Preserve non-empty lines
            # print(inside_code_block, line)
            result.append(line)

    return "\n".join(result)


# # Example usage
# input_text = """
# Some text outside the code block.

# ```python
# # This is a sample Python code block
# print("Hello, world!")
# a = 5 + 3


In [17]:
passed = 0
syntaxError = 0
AssertionError = 0
KeyError = 0
ValueError = 0
NameError = 0
id = 0

for row in tqdm(gsm8k_dataset['train'], desc="gsm8k dataset", unit="sample"):
    question = row['question']
    answer = row['answer']
    solution = row['code']
    id = id + 1

    print(f"{id} :")

    if solution is None:
        print(f"Failed: {id} : {solution}")
        continue

    # if id == 580 or id==890:
    #     continue

    # print(solution)
    combined_code = extract_code(solution)
    # print(combined_code)
    # break

    result = execute_code(combined_code)

    # if result == True:
    #     passed += 1
    # else:


print(f"Passed: {passed}/{len(gsm8k_dataset['train'])}")


gsm8k dataset:   0%|          | 0/1001 [00:00<?, ?sample/s]

1 :
Total amount of money made per week by Janet: 48
2 :
It takes 3.0 bolts of fabric.
3 :
4 :
Total distance run by James: 180 meters
Time taken by James to run 100 meters: 57.0 seconds
5 :
To determine how many cups of feed Wendi needs to give her chickens in the final meal of the day, we can use the following steps:
1. Determine the initial number of chickens.
Chicken count: 20
2. Determine the amount of feed given in the morning and afternoon.
Morning feed: 15 cups
Afternoon feed: 25 cups
3. Calculate the total amount of feed given in the morning and afternoon.
Total feed: 40 cups
4. Double the initial number of chickens.
Doubled chicken count: 40
5. Subtract one from the number of chickens after a dog eats one of the chickens.
6 :
Total cost: 32
7 :
Total number of sheep in all three cities: 80.0
8 :
Time for the first file: 100 minutes
Time for the second file: 60 minutes
Time for the third file: 20 minutes
Total download time: 3 hours
9 :
10 :
Total earnings for 45 hours worked 

gsm8k dataset:   7%|▋         | 72/1001 [00:02<00:37, 24.86sample/s]

73 :
Total amount raised: $221
74 :
75 :
Total cost: 19
76 :
Average square footage of each level: 5.694444444444445
77 :
Total number of bags of dog food used in the first year: 4
78 :
79 :
80 :
Total cost of grooming: 550.0
81 :
There are 14.0 liters of water left.
82 :
17
83 :
84 :
Total Thorns: 600
85 :
Number of wins: 22
86 :
Total number of quilt blocks: 12.363636363636363
87 :
Number of intervals:  3
Total number of rings:  22
88 :
After 3 years of service, Sylvie's new salary would be $780.0
89 :
Total Earnings: 1108800.0
90 :
Number of guests who will receive gift bags: 12
Total cost of gift bags for the guests who will receive them: 24
Total cost of gift bags for all guests: 384
Total amount Monica will make: 334
91 :
Total weight of potato salad needed: 225.0 lbs
92 :
93 :
Jeremy's age is 100
Amy's age is 33
Jackson's age is 33
Corey's age is 34
James's age is 10
94 :
Gerald's new time running the 400-meter hurdles with his improved diet is 35 seconds.
The difference in time

gsm8k dataset:  15%|█▌        | 154/1001 [00:04<00:24, 33.91sample/s]

...

851 :
0
852 :
853 :
854 :
855 :
12 + 36 + 96 = 144
856 :
['mom_age', '=', '3', 'brother_age', '-', '1']
857 :
858 :
Total amount of water passing through the river at this point: 10000 gallons
859 :
Initially, there are 70 people.
Each day, 60 more people become infected.
After one hour, there will be 2 new cases.
After two hours, there will be 62 new cases.
After three hours, there will be 122 new cases.
After four hours, there will be 182 new cases.
After five hours, there will be 242 new cases.
There will be a total of 242 infected people after five hours.
860 :
Julieta and Rafael received 38.0 as tips together.
Donna and Marcia also purchased 400 pencils between them.
861 :
5 oranges remained.
862 :
863 :
Total Amount Raised: 800
864 :
Total number of coins Gretchen has: 100.0
865 :
866 :
867 :
868 :
The weight of the sweet potatoes is 10 pounds.
The weight of the carrots is 7 pounds.
869 :
Total cost of jello: $450.0
870 :
Total cost of the phones: 3500
Total amount of money the s

gsm8k dataset:  91%|█████████▏| 914/1001 [00:45<00:04, 17.97sample/s]

915 :
Jon needs 3 tune-ups
916 :
Lawrence works 7.666666666666667 hours per day.
917 :
Enter Julia's initial allowance: 


gsm8k dataset: 100%|██████████| 1001/1001 [00:47<00:00, 21.02sample/s]

918 :
There are 12 pieces of fruit left.
919 :
Total Students: 6
Basketball Students: 10
Volleyball Students: 8
Students Playing Both Games: 2
920 :
the amount of money raised by the class: 5
The class can raise 5 from the profit of selling lollipops.
921 :
922 :
923 :
924 :
Mint Tea Per Row: 9
925 :
Total number of trips needed: 24
Total time spent on the trip: 8
926 :
Day 1 : Tank level =  5000.0
Day 2 : Tank level =  5000.0
Day 3 : Tank level =  5000.0
Day 4 : Tank level =  5000.0
Day 5 : Tank level =  5000.0
Day 6 : Tank level =  5000.0

Final tank level = 5000.0
927 :
16
928 :
Total Calories of Pastries: 34000.0
929 :
Error: No data found for Wednesday
930 :
The bus went 60 miles further than the car.
931 :
932 :
933 :
22
934 :
Total number of boxes donated: 60
Number of tables needed: 30
Total amount of money spent: $600
Number of boxes that can be packed: 60
Total number of new tables needed: 15
935 :
936 :
For a month with 30 days, the difference in revenue between Widgeteer 30